In [1]:
!pip3 install qcircuits

  Preparing metadata (setup.py) ... done
  Created wheel for qcircuits: filename=QCircuits-0.6.0-py3-none-any.whl size=17263 sha256=6f8ba8e4fbb2e38d846f6e29d9780f2b86469481ef089235220c23d41dc45729
  Stored in directory: /Users/adrianariton/Library/Caches/pip/wheels/9c/da/bc/f080b41e92bfbbd796b03c2f0962ec3ab488d8a0b122a2a4e2
Successfully built qcircuits

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3.11 -m pip install --upgrade pip


If the instructions presented in the README.md file were correctly followed, QCircuits should be correctly installed, and we can begin using the library!

In [2]:
import qcircuits as qc

Let's see how writing code with **qcircuits** actially looks like!

This function is the well known quantum teleportation algorithm!

In [3]:
def quantum_teleportation(alice_state):
    # Get operators we will need
    CNOT = qc.CNOT()
    H = qc.Hadamard()
    X = qc.PauliX()
    Z = qc.PauliZ()

    # The prepared, shared Bell state
    bell = qc.bell_state(0, 0)
    
    # The whole state vector
    state = alice_state * bell

    # Apply CNOT and Hadamard gate
    state = CNOT(state, qubit_indices=[0, 1])
    state = H(state, qubit_indices=[0])

    # Measure the first two bits
    # The only uncollapsed part of the state vector is Bob's
    M1, M2 = state.measure(qubit_indices=[0, 1], remove=True)

    # Apply X and/or Z gates to third qubit depending on measurements
    if M2:
        state = X(state)
    if M1:
        state = Z(state)

    return state

![qtelimg][def]

[def]: teleport.png

As you can see, writing code with QCircuits is fairly easy and elegant and therefore, beginer friendly!

Let's take the function step by step!

## Let's try to understand a little how qcircuit simulates quantum circuits


1. First of all we define our operators:

    ```python
        CNOT = qc.CNOT()
        H = qc.Hadamard()
        X = qc.PauliX()
        Z = qc.PauliZ()
    ```
2. We initialize our needed state to the bell state:

    ![bell state](https://upload.wikimedia.org/wikipedia/commons/f/fc/The_Hadamard-CNOT_transform_on_the_zero-state.png)

    ```python
        bell = qc.bell_state(0, 0)
    ```
3. We multiply (tensorial multiplication) the alice_state and out newly generated bell pair

    ```python
        state = alice_state * bell
    ```
    This is mandatory, because qcircuits needs to keep track of all the qubits in a single vector type.
    The qubits positions will be determined as such:
    $$| \psi \rangle = q_0$$
    The EPR pair:$$| \beta_{00} \rangle = q_{[1,2]}$$

4. Afterwards, the inverse of the entanglement is performed

    ```python
        state = CNOT(state, qubit_indices=[0, 1])
        state = H(state, qubit_indices=[0])
    ```

5. And then the state is measured

    ```python
        M1, M2 = state.measure(qubit_indices=[0, 1], remove=True)

        # Apply X and/or Z gates to third qubit depending on measurements
        if M2:
            state = X(state)
        if M1:
            state = Z(state)
    ```

Let's run it see how it does!

In [4]:
if __name__ == '__main__':
    # Alice's original state to be teleported to Bob
    alice = qc.qubit(theta=1.5, phi=0.5, global_phase=0.2)

    # Bob's state after quantum teleportation
    bob = quantum_teleportation(alice)

    print('Original state:', alice)
    print('\nTeleported state:', bob)

Original state: 1-qubit state. Tensor:
[0.71710381+0.14536414j 0.52134608+0.43912375j]

Teleported state: 1-qubit state. Tensor:
[0.71710381+0.14536414j 0.52134608+0.43912375j]


As you can see, the teleportation function produced the exact same output!



> But why doesn't Alice's state get destroyed?? Seems like the simulation is wrong somehow...

I hear you say...



A: Well turns out that qcircuits can take snapshots of states and keep them as a copy, and the `alice` parameter is not interpreted as a refference and is therefore copied!